In [11]:
import pandas as pd
from backtesting import Backtest, Strategy

# Load your CSV file
data = pd.read_csv('check1.csv', parse_dates=True, index_col='Datetime')

class CustomStrategy(Strategy):
    def init(self):
        # Store the target sell prices for each trade
        self.target_sell_prices = []

    def next(self):
        if not self.position:
            # Buy signal logic
            target_sell_price = self.data.Close[-1] + (self.data.Prediction[-1] * ((self.data.Close[-1] / 100) * 0.024))
            self.buy(sl=None)
            self.target_sell_prices.append(target_sell_price)
        elif self.position:
            # Check if current price meets the target sell price
            if self.data.Close[-1] >= self.target_sell_prices[-1]:
                self.position.close()
                self.target_sell_prices.pop()  # Remove the target sell price for the closed position

# Define initial cash and commission
initial_cash = 100000  # Initial cash amount
commission = 0.000  # Commission rate of 0.1%

# Run the backtest
bt = Backtest(data, CustomStrategy, cash=initial_cash, commission=commission)
stats = bt.run()

# Display the results
print(stats)
#bt.plot()


Start                     2024-07-19 04:59...
End                       2024-08-08 04:59...
Duration                     20 days 00:00:00
Exposure Time [%]                   99.993056
Equity Final [$]                     92032.39
Equity Peak [$]                      105172.4
Return [%]                           -7.96761
Buy & Hold Return [%]              -11.530572
Return (Ann.) [%]                  -82.808792
Volatility (Ann.) [%]                6.011429
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -19.55031
Avg. Drawdown [%]                   -0.445584
Max. Drawdown Duration        9 days 15:44:00
Avg. Drawdown Duration        0 days 05:19:00
# Trades                                  111
Win Rate [%]                        99.099099
Best Trade [%]                       0.347249
Worst Trade [%]                    -18.798281
Avg. Trade [%]                    

/home/nitin/Downloads/Project/myenv/lib/python3.12/site-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '5T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/home/nitin/Downloads/Project/myenv/lib/python3.12/site-packages/backtesting/_plotting.py:126: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample(freq, label='right').agg(OHLCV_AGG).dropna()
/home/nitin/Downloads/Project/myenv/lib/python3.12/site-packages/backtesting/_plotting.py:136: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  equity_data = equity_data.resample(freq, label='right').agg(_EQUITY_AGG).dropna(how='all')
/home/nitin/Downloads/Project/myenv/lib/python3.12/site-packages/backtesting/_plotting.py:153: FutureWarning: 'T' is deprecated and will be removed in a 

TypeError: Index.get_loc() got an unexpected keyword argument 'method'

In [20]:
import pandas as pd
from backtesting import Backtest, Strategy

class CustomTradingStrategy(Strategy):
    def init(self):
        self.close = self.data.Close
        self.prediction = self.data.Prediction
        self.positions = {}
        self.initial_cash = self.equity / 15

    def next(self):
        prediction = self.prediction
        close_price = self.close

        # Check if there are any positions to sell
        for key, position in list(self.positions.items()):
            if close_price >= position['target_sell_price']:
                self.sell(size=position['amount'])
                print(f"Sold position: {key} at {close_price} | Target: {position['target_sell_price']}")
                del self.positions[key]

        # Calculate available cash
        open_position_value = sum(position['amount'] for position in self.positions.values())
        cash = self.equity - open_position_value

        # Buy if the conditions are met
        if cash >= self.initial_cash and prediction in [1, 2, 3, 4, 5] and len(self.positions) < 15:
            target_sell_price = close_price + (prediction * ((close_price / 100) * 0.042))
            self.buy(size=self.initial_cash)
            trade_key = f"{len(self.positions)}"
            self.positions[trade_key] = {
                'amount': self.initial_cash,
                'target_sell_price': target_sell_price
            }
            print(f"Bought position: {trade_key} at {close_price} | Target: {target_sell_price}")

# Load your data
data = pd.read_csv('check1.csv', parse_dates=True, index_col='Datetime')

# Backtest the strategy
bt = Backtest(data, CustomTradingStrategy, cash=100000, commission=.001)
stats = bt.run()
#bt.plot()

# Print the statistics
print(stats)


Start                     2024-07-19 04:59...
End                       2024-08-08 04:59...
Duration                     20 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                     100000.0
Equity Peak [$]                      100000.0
Return [%]                                0.0
Buy & Hold Return [%]              -11.530572
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [28]:
57047.07-56760.0

287.0699999999997

In [17]:
from PyPDF2 import PdfReader, PdfWriter
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Frame, PageTemplate
from reportlab.lib.units import inch
from io import BytesIO

# Path to the uploaded PDF file
input_path = "New Club Proposal 1.pdf"
output_path = "New Club Proposal 1_Cleaned.pdf"

# Function to remove extra spaces from the text
def clean_text(text):
    lines = text.splitlines()
    cleaned_lines = [line.strip() for line in lines if line.strip()]
    return " ".join(cleaned_lines)  # Join lines into a single paragraph

# Read the input PDF
input_pdf = PdfReader(open(input_path, "rb"))
output_pdf = PdfWriter()

# Extract and clean text from all pages
full_text = ""
for page_num in range(len(input_pdf.pages)):
    page = input_pdf.pages[page_num]
    text = page.extract_text()
    cleaned_text = clean_text(text)
    full_text += cleaned_text + " "  # Combine text from all pages into a single string

# Use ReportLab to create a new PDF with the cleaned and combined text
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=letter)
styles = getSampleStyleSheet()
style = styles['Normal']

# Create a Frame to define the area where the text will be placed
frame = Frame(inch, inch, 6.5 * inch, 9 * inch, showBoundary=0)

# Define a PageTemplate with the created Frame
template = PageTemplate(id='cleaned', frames=[frame])
doc.addPageTemplates([template])

# Build the document
story = [Paragraph(full_text, style)]
doc.build(story)

packet.seek(0)
new_pdf = PdfReader(packet)

# Add the generated pages to the output PDF
for page_num in range(len(new_pdf.pages)):
    output_pdf.add_page(new_pdf.pages[page_num])

# Save the cleaned PDF to a new file
with open(output_path, "wb") as output_file:
    output_pdf.write(output_file)

output_path


'New Club Proposal 1_Cleaned.pdf'

In [3]:
import fitz  # PyMuPDF

def remove_extra_space(input_pdf, output_pdf):
    # Open the input PDF
    doc = fitz.open(input_pdf)
    
    # Iterate through each page
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        
        # Get the text blocks (rectangles) on the page
        blocks = page.get_text("blocks")
        
        # Iterate through each block to adjust formatting
        for block in blocks:
            rect = fitz.Rect(block[:4])  # The rectangle of the block
            text = block[4]  # The text in the block
            
            # Define the new position of the block
            new_rect = fitz.Rect(rect.x0, rect.y0, rect.x1, rect.y1 - 5)  # Adjusting the y1 coordinate
            
            # Add the adjusted text back to the page within the new rectangle
            page.insert_textbox(new_rect, text, fontsize=block[5])
    
    # Save the modified PDF
    doc.save(output_pdf)
    doc.close()

# Specify input and output PDF file paths
input_pdf = "New Club Proposal 1.pdf"
output_pdf = "output.pdf"

# Remove extra space from the PDF
remove_extra_space(input_pdf, output_pdf)


In [6]:
from docx import Document

# Create a new Document
doc = Document()

# Add Title
doc.add_heading('Quantitative Finance Club Proposal', 0)

# Add Subtitle
doc.add_heading('STUDENT ACTIVITIES – IIIT Bhopal', level=1)

# Add Contact Information Section
doc.add_heading('CONTACT INFORMATION', level=1)
doc.add_paragraph('Proposed Name of Club:\nQuantitative Finance Club')
doc.add_paragraph('Proposed Faculty Coordinator:\nDr. Saurabh Jain')
doc.add_paragraph('Student(s) Submitting the Proposal:\nNitin Yadav - 22U01025@iiitbhopal.ac.in\nSuryansh Yagnik - 22U01037@iiitbhopal.ac.in')

# Add Type of Club Section
doc.add_heading('Type of Club: (select up to three)', level=1)
doc.add_paragraph('[ ] Academic\n[ ] Activism/Advocacy\n[ ] Arts/Media\n[ ] Business/Innovation\n[ ] Competition\n[ ] De-Stress/Mindfulness\n[ ] Cultural/International\n[ ] Games/Gaming\n[ ] Government/Political\n[ ] Publication\n[ ] Recreation\n[ ] Service/Outreach\n[ ] Special Interest\n[ ] STEM')

# Add Verification Section
doc.add_heading('VERIFICATION', level=1)
doc.add_paragraph('I certify that this proposal complies with all the “General Club Criteria”\n\n_________________________\n(Student Leader Signature)\n_________________________\n(Date)\n\n_________________________\n(Proposed Coordinator Signature)\n_________________________\n(Date)')

# Add Proposal Section
doc.add_heading('YOUR PROPOSAL - Who, What, Where, When, Why, and How?', level=1)

# Add Subsection for Purpose
doc.add_heading('WHAT is the purpose of your club? (mission/goals/objectives)', level=2)
doc.add_paragraph("The Quantitative Finance Club at IIIT Bhopal aims to foster a deep understanding of quantitative finance by integrating finance, mathematics, and computer science. Our goals are to enhance technical skills, promote innovative research, and build industry connections. We provide hands-on projects, educational resources, and organize events to cultivate a collaborative learning environment for aspiring finance professionals.")

# Add Subsection for Target Audience
doc.add_heading('WHO will this club serve? Who is your target audience? Be specific.', level=2)
doc.add_paragraph("The Quantitative Finance Club at IIIT Bhopal serves all students at IIIT Bhopal with a keen interest in finance, mathematics, computer science, and related fields. Our target audience includes finance enthusiasts, mathematics and statistics students, computer science students, and economics students. We welcome both beginners eager to learn and advanced learners aiming to deepen their expertise. Membership is open to all IIIT Bhopal students, fostering a diverse and inclusive environment for collaborative learning, skill enhancement, and professional growth.")

# Add Subsection for Meeting Space
doc.add_heading('WHERE will you plan to meet? What space is needed?', level=2)
doc.add_paragraph("The Quantitative Finance Club at IIIT Bhopal plans to meet in designated classrooms or seminar halls within the campus equipped with necessary facilities for presentations and discussions. For workshops and hands-on sessions, we will utilize computer labs with internet access and relevant software. Additionally, we may use meeting rooms or study spaces for smaller group activities and project collaborations. These spaces should provide a conducive environment for learning, collaboration, and hosting guest speakers or industry professionals.")

# Add Subsection for Addressing Needs
doc.add_heading('WHY? Does this club address a new need? How is it different from the other already established clubs/groups/activities offered at IIIT Bhopal?', level=2)
doc.add_paragraph("The Quantitative Finance Club addresses the growing demand for skills in quantitative finance, bridging the gap between theoretical knowledge and practical application in finance, mathematics, and computer science. Unlike other clubs, our focus is specifically on integrating these disciplines to develop advanced financial models, trading strategies, and risk management techniques. We offer unique hands-on projects, research opportunities, and industry connections, providing a specialized platform for students to gain expertise in quantitative finance, which is not covered by existing clubs or activities at IIIT Bhopal.")

# Add Subsection for Ensuring Success
doc.add_heading('What steps will you take to ensure this club is successful? (please be specific)', level=2)
doc.add_paragraph("1. **Set Clear Goals:** Define specific measurable objectives for the trial period.\n2. **Recruit and Engage:** Attract motivated members and ensure active participation.\n3. **Organize Regular Activities:** Schedule meetings, workshops, and guest lectures.\n4. **Involve Mentors:** Collaborate with faculty and industry experts for guidance.\n5. **Gather Feedback:** Implement a system to collect and act on member input.\n6. **Promote and Document:** Publicize events, track progress, and prepare for leadership transitions.")

# Add Subsection for Special Equipment
doc.add_heading('OTHER: List any special equipment needed if any.', level=2)
doc.add_paragraph("**Computers:** For financial modeling, algorithm development, and data analysis.\n**Projectors:** For presentations and workshops.\n**Whiteboards/Flipcharts:** For brainstorming and collaborative sessions.\n**Audio-Visual Equipment:** For guest lectures and online meetings.")

# Add Subsection for Leadership Structure
doc.add_heading('Identify the leadership structure for your club (titles, positions, responsibilities). Be sure to include the selection criteria as well.', level=2)
doc.add_paragraph("1. **Lead**\n   - **Responsibilities:**\n     - Oversee the club's overall functioning and represent it in official matters.\n     - Coordinate with faculty advisors and the college administration.\n     - Lead the development and execution of the club's vision and goals.\n   - **Selection Criteria:**\n     - Proven leadership skills and expertise in quantitative finance.\n     - Strong communication and organizational abilities.\n\n2. **Research and Development Lead**\n   - **Responsibilities:**\n     - Lead the club's research initiatives and explore new topics in quantitative finance.\n     - Organize study groups and research projects among members.\n     - Collaborate with faculty and industry professionals for advanced learning.\n   - **Selection Criteria:**\n     - In-depth knowledge of quantitative finance and research experience.\n     - Ability to mentor and guide members in research efforts.\n\n3. **Technical and Analytics Lead**\n   - **Responsibilities:**\n     - Manage the club’s technical projects including algorithm development and data analysis.\n     - Oversee coding workshops and technical training sessions.\n     - Ensure that members are equipped with the necessary technical skills.\n   - **Selection Criteria:**\n     - Strong background in programming, data analysis, and quantitative methods.\n     - Experience in leading technical projects or teams.\n\n4. **Management and Outreach**\n   - **Responsibilities:**\n     - Plan and execute events such as workshops, webinars, and seminars.\n     - Coordinate with speakers, facilitators, and participants.\n     - Promote events and manage attendance.\n     - Promote the club’s activities within and outside the college.\n   - **Selection Criteria:**\n     - Strong event management, communication, and marketing skills.\n     - Ability to collaborate with industry professionals and manage social media.")

# Add Peer Support Section
doc.add_heading('PEER SUPPORT', level=1)
doc.add_paragraph("Please collect at least 12 hand-written signatures of students who are interested in participating in the proposed club. A variety of students and grade levels may help strengthen your proposal but is not required.")
table = doc.add_table(rows=1, cols=4)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Name'
hdr_cells[1].text = 'Course'
hdr_cells[2].text = 'Student ID'
hdr_cells[3].text = 'Signature'

for i in range(1, 21):
    row_cells = table.add_row().cells
    row_cells[0].text = str(i) + '.'

# Add Footer
section = doc.sections[0]
footer = section.footer
footer_para = footer.paragraphs[0]
footer_para.text = "IIIT Bhopal - Updated August 2024"

# Save the document
output_path = "Quantitative_Finance_Club_Proposal_IIIT_Bhopal.docx"
doc.save(output_path)

output_path


'Quantitative_Finance_Club_Proposal_IIIT_Bhopal.docx'

In [7]:
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.section import WD_SECTION
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

# Create a new Document
doc = Document()

# Add Title
title = doc.add_heading('Quantitative Finance Club Proposal', 0)
title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Add Subtitle
subtitle = doc.add_heading('STUDENT ACTIVITIES – IIIT Bhopal', level=1)
subtitle.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Add Contact Information Section
doc.add_heading('CONTACT INFORMATION', level=1)
doc.add_paragraph('Proposed Name of Club:\nQuantitative Finance Club')
doc.add_paragraph('Proposed Faculty Coordinator:\nDr. Saurabh Jain')
doc.add_paragraph('Student(s) Submitting the Proposal:\nNitin Yadav - 22U01025@iiitbhopal.ac.in\nSuryansh Yagnik - 22U01037@iiitbhopal.ac.in')

# Add Type of Club Section
doc.add_heading('Type of Club: (select up to three)', level=1)
doc.add_paragraph(
    '[ ] Academic\n[ ] Activism/Advocacy\n[ ] Arts/Media\n[ ] Business/Innovation\n[ ] Competition\n'
    '[ ] De-Stress/Mindfulness\n[ ] Cultural/International\n[ ] Games/Gaming\n[ ] Government/Political\n'
    '[ ] Publication\n[ ] Recreation\n[ ] Service/Outreach\n[ ] Special Interest\n[ ] STEM'
)

# Add Verification Section
doc.add_heading('VERIFICATION', level=1)
verification_paragraph = doc.add_paragraph(
    'I certify that this proposal complies with all the “General Club Criteria”\n\n'
    '_________________________\n(Student Leader Signature)\n_________________________\n(Date)\n\n'
    '_________________________\n(Proposed Coordinator Signature)\n_________________________\n(Date)'
)

# Add Proposal Section
doc.add_heading('YOUR PROPOSAL - Who, What, Where, When, Why, and How?', level=1)

# Add Subsection for Purpose
doc.add_heading('WHAT is the purpose of your club? (mission/goals/objectives)', level=2)
doc.add_paragraph(
    "The Quantitative Finance Club at IIIT Bhopal aims to foster a deep understanding of quantitative finance by "
    "integrating finance, mathematics, and computer science. Our goals are to enhance technical skills, promote "
    "innovative research, and build industry connections. We provide hands-on projects, educational resources, and "
    "organize events to cultivate a collaborative learning environment for aspiring finance professionals."
)

# Add Subsection for Target Audience
doc.add_heading('WHO will this club serve? Who is your target audience? Be specific.', level=2)
doc.add_paragraph(
    "The Quantitative Finance Club at IIIT Bhopal serves all students at IIIT Bhopal with a keen interest in finance, "
    "mathematics, computer science, and related fields. Our target audience includes finance enthusiasts, mathematics "
    "and statistics students, computer science students, and economics students. We welcome both beginners eager to "
    "learn and advanced learners aiming to deepen their expertise. Membership is open to all IIIT Bhopal students, "
    "fostering a diverse and inclusive environment for collaborative learning, skill enhancement, and professional growth."
)

# Add Subsection for Meeting Space
doc.add_heading('WHERE will you plan to meet? What space is needed?', level=2)
doc.add_paragraph(
    "The Quantitative Finance Club at IIIT Bhopal plans to meet in designated classrooms or seminar halls within the "
    "campus equipped with necessary facilities for presentations and discussions. For workshops and hands-on sessions, "
    "we will utilize computer labs with internet access and relevant software. Additionally, we may use meeting rooms "
    "or study spaces for smaller group activities and project collaborations. These spaces should provide a conducive "
    "environment for learning, collaboration, and hosting guest speakers or industry professionals."
)

# Add Subsection for Addressing Needs
doc.add_heading('WHY? Does this club address a new need? How is it different from the other already established clubs/groups/activities offered at IIIT Bhopal?', level=2)
doc.add_paragraph(
    "The Quantitative Finance Club addresses the growing demand for skills in quantitative finance, bridging the gap "
    "between theoretical knowledge and practical application in finance, mathematics, and computer science. Unlike other "
    "clubs, our focus is specifically on integrating these disciplines to develop advanced financial models, trading strategies, "
    "and risk management techniques. We offer unique hands-on projects, research opportunities, and industry connections, "
    "providing a specialized platform for students to gain expertise in quantitative finance, which is not covered by existing "
    "clubs or activities at IIIT Bhopal."
)

# Add Subsection for Ensuring Success
doc.add_heading('What steps will you take to ensure this club is successful? (please be specific)', level=2)
doc.add_paragraph(
    "1. **Set Clear Goals:** Define specific measurable objectives for the trial period.\n"
    "2. **Recruit and Engage:** Attract motivated members and ensure active participation.\n"
    "3. **Organize Regular Activities:** Schedule meetings, workshops, and guest lectures.\n"
    "4. **Involve Mentors:** Collaborate with faculty and industry experts for guidance.\n"
    "5. **Gather Feedback:** Implement a system to collect and act on member input.\n"
    "6. **Promote and Document:** Publicize events, track progress, and prepare for leadership transitions."
)

# Add Subsection for Special Equipment
doc.add_heading('OTHER: List any special equipment needed if any.', level=2)
doc.add_paragraph(
    "**Computers:** For financial modeling, algorithm development, and data analysis.\n"
    "**Projectors:** For presentations and workshops.\n"
    "**Whiteboards/Flipcharts:** For brainstorming and collaborative sessions.\n"
    "**Audio-Visual Equipment:** For guest lectures and online meetings."
)

# Add Subsection for Leadership Structure
doc.add_heading('Identify the leadership structure for your club (titles, positions, responsibilities). Be sure to include the selection criteria as well.', level=2)
doc.add_paragraph(
    "1. **Lead**\n"
    "   - **Responsibilities:**\n"
    "     - Oversee the club's overall functioning and represent it in official matters.\n"
    "     - Coordinate with faculty advisors and the college administration.\n"
    "     - Lead the development and execution of the club's vision and goals.\n"
    "   - **Selection Criteria:**\n"
    "     - Proven leadership skills and expertise in quantitative finance.\n"
    "     - Strong communication and organizational abilities.\n\n"
    "2. **Research and Development Lead**\n"
    "   - **Responsibilities:**\n"
    "     - Lead the club's research initiatives and explore new topics in quantitative finance.\n"
    "     - Organize study groups and research projects among members.\n"
    "     - Collaborate with faculty and industry professionals for advanced learning.\n"
    "   - **Selection Criteria:**\n"
    "     - In-depth knowledge of quantitative finance and research experience.\n"
    "     - Ability to mentor and guide members in research efforts.\n\n"
    "3. **Technical and Analytics Lead**\n"
    "   - **Responsibilities:**\n"
    "     - Manage the club’s technical projects including algorithm development and data analysis.\n"
    "     - Oversee coding workshops and technical training sessions.\n"
    "     - Ensure that members are equipped with the necessary technical skills.\n"
    "   - **Selection Criteria:**\n"
    "     - Strong background in programming, data analysis, and quantitative methods.\n"
    "     - Experience in leading technical projects or teams.\n\n"
    "4. **Management and Outreach**\n"
    "   - **Responsibilities:**\n"
    "     - Plan and execute events such as workshops, webinars, and seminars.\n"
    "     - Coordinate with speakers, facilitators, and participants.\n"
    "     - Promote events and manage attendance.\n"
    "     - Promote the club’s activities within and outside the college.\n"
    "   - **Selection Criteria:**\n"
    "     - Strong event management, communication, and marketing skills.\n"
    "     - Ability to collaborate with industry professionals and manage social media."
)

# Add Peer Support Section
doc.add_heading('PEER SUPPORT', level=1)
peer_support_paragraph = doc.add_paragraph(
    "Please collect at least 12 hand-written signatures of students who are interested in participating in the proposed club. "
    "A variety of students and grade levels may help strengthen your proposal but is not required."
)

# Add table for signatures
table = doc.add_table(rows=1, cols=4)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Name'
hdr_cells[1].text = 'Course'
hdr_cells[2].text = 'Student ID'
hdr_cells[3].text = 'Signature'

for i in range(1, 21):
    row_cells = table.add_row().cells
    row_cells[0].text = str(i) + '.'

# Add Footer
section = doc.sections[0]
footer = section.footer
footer_para = footer.paragraphs[0]
footer_para.text = "IIIT Bhopal - Updated August 2024"
footer_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Save the document
output_path = "Quantitative_Finance_Club_Proposal_IIIT_Bhopal.docx"
doc.save(output_path)

output_path


'Quantitative_Finance_Club_Proposal_IIIT_Bhopal.docx'

In [36]:
two = 57019.62-56864.0
three = 57075.53 - 56864.0
four = 57146.04- 56864.0
five = 57216.55- 56864.0
six = 57287.07 - 56864.0
print(two)
print(three)
print(four)
print(five)
print(six)

155.62000000000262
211.52999999999884
282.0400000000009
352.5500000000029
423.0699999999997


In [35]:
56864.0 + (6 * ((56864.0 / 100) * 0.124))

57287.06816

In [1]:
from binance.client import Client

In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv()
api_secret_bin = os.getenv('secret_key_bin')
api_key_bin = os.getenv('api_key_bin')
client = Client(api_key_bin, api_secret_bin, testnet=True)

In [3]:
def get_latest_prices(client):
    prices = client.get_all_tickers()
    price_dict = {price['symbol']: float(price['price']) for price in prices}
    return price_dict

In [4]:
client.get_my_trades(symbol = 'BTCUSDT')

[{'symbol': 'BTCUSDT',
  'id': 894862,
  'orderId': 3416395,
  'orderListId': -1,
  'price': '56928.01000000',
  'qty': '0.00660000',
  'quoteQty': '375.72486600',
  'commission': '0.00000000',
  'commissionAsset': 'BTC',
  'time': 1725979849898,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 894863,
  'orderId': 3416395,
  'orderListId': -1,
  'price': '56934.05000000',
  'qty': '0.00352000',
  'quoteQty': '200.40785600',
  'commission': '0.00000000',
  'commissionAsset': 'BTC',
  'time': 1725979849898,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 894864,
  'orderId': 3416395,
  'orderListId': -1,
  'price': '56934.07000000',
  'qty': '0.00782000',
  'quoteQty': '445.22442740',
  'commission': '0.00000000',
  'commissionAsset': 'BTC',
  'time': 1725979849898,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 894865,
  'orderId': 3416395,
  'orderL

In [5]:
def get_holdings(client, base_currency='USDT'):
    account_info = client.get_account()
    balances = account_info['balances']
    prices = get_latest_prices(client)
    
    holdings_info = []
    for balance in balances:
        asset = balance['asset']
        free_balance = float(balance['free'])
        locked_balance = float(balance['locked'])
        total_balance = free_balance + locked_balance

        if total_balance > 0:
            symbol = asset + base_currency
            current_price = prices.get(symbol, None)
            
            if current_price:
                market_value = total_balance * current_price
                # Assuming we have a way to get the cost basis; using current price for now as placeholder
                # In practice, this would be the average purchase price or an actual value from your records
                cost_basis = current_price  # Placeholder: You should replace this with the actual cost basis
                profit_loss = market_value - (total_balance * cost_basis)
                
                holdings_info.append({
                    'symbol': asset,
                    'quantity': total_balance,
                    'current_price': current_price,
                    'market_value': market_value,
                    'profit_loss': profit_loss
                })
            else:
                print(f"Price for {symbol} not found, skipping.")

    return holdings_info


get_holdings(client)

Price for USDTUSDT not found, skipping.
Price for TRYUSDT not found, skipping.
Price for ZARUSDT not found, skipping.
Price for IDRTUSDT not found, skipping.
Price for UAHUSDT not found, skipping.
Price for DAIUSDT not found, skipping.
Price for BRLUSDT not found, skipping.
Price for PLNUSDT not found, skipping.
Price for RONUSDT not found, skipping.
Price for ARSUSDT not found, skipping.
Price for JPYUSDT not found, skipping.
Price for MXNUSDT not found, skipping.
Price for CZKUSDT not found, skipping.
Price for COPUSDT not found, skipping.


[{'symbol': 'ETH',
  'quantity': 1.0,
  'current_price': 2551.66,
  'market_value': 2551.66,
  'profit_loss': 0.0},
 {'symbol': 'BTC',
  'quantity': 1.07187,
  'current_price': 63047.9,
  'market_value': 67579.15257300001,
  'profit_loss': 0.0},
 {'symbol': 'LTC',
  'quantity': 7.0,
  'current_price': 65.76,
  'market_value': 460.32000000000005,
  'profit_loss': 0.0},
 {'symbol': 'BNB',
  'quantity': 1.0,
  'current_price': 582.4,
  'market_value': 582.4,
  'profit_loss': 0.0},
 {'symbol': 'TRX',
  'quantity': 3343.0,
  'current_price': 0.1517,
  'market_value': 507.1331,
  'profit_loss': 0.0},
 {'symbol': 'XRP',
  'quantity': 894.0,
  'current_price': 0.5832,
  'market_value': 521.3808,
  'profit_loss': 0.0},
 {'symbol': 'NEO',
  'quantity': 52.0,
  'current_price': 10.28,
  'market_value': 534.56,
  'profit_loss': 0.0},
 {'symbol': 'QTUM',
  'quantity': 229.0,
  'current_price': 2.51,
  'market_value': 574.79,
  'profit_loss': 0.0},
 {'symbol': 'EOS',
  'quantity': 1088.0,
  'current

In [6]:
def get_total_cash(client, base_currency='USDT'):
    # Get balance for the specified asset
    asset_balance = client.get_asset_balance(asset='BTC')
    ticker = client.get_symbol_ticker(symbol='BTCUSDT')
    price = float(ticker['price'])
    
    # Extract the free balance (available balance)
    total_cash = float(asset_balance['free'])
    
    return total_cash*price

get_total_cash(client)

67579.15257300001

In [7]:
def get_total_equity(client, base_currency='USDT'):
    holdings = get_holdings(client, base_currency=base_currency)
    
    total_equity = 0.0
    for holding in holdings:
        total_equity += holding['market_value']
    
    return total_equity

ignore_symbols = ['USDT', 'TRY', 'ZAR', 'IDRT', 'UAH', 'DAI', 'BRL', 'PLN', 'RON', 'ARS', 'JPY', 'MXN', 'CZK']

get_total_equity(client)

Price for USDTUSDT not found, skipping.
Price for TRYUSDT not found, skipping.
Price for ZARUSDT not found, skipping.
Price for IDRTUSDT not found, skipping.
Price for UAHUSDT not found, skipping.
Price for DAIUSDT not found, skipping.
Price for BRLUSDT not found, skipping.
Price for PLNUSDT not found, skipping.
Price for RONUSDT not found, skipping.
Price for ARSUSDT not found, skipping.
Price for JPYUSDT not found, skipping.
Price for MXNUSDT not found, skipping.
Price for CZKUSDT not found, skipping.
Price for COPUSDT not found, skipping.


359382.10992214014

In [8]:
from datetime import datetime
def fetch_all_transactions(client):
    # Fetch all symbols that have been traded by the user
    account_info = client.get_account()
    all_symbols = ['BTCUSDT']#[balance['asset'] + 'USDT' for balance in account_info['balances'] if float(balance['free']) + float(balance['locked']) > 0 and balance['asset'] not in ignore_symbols]

    transactions = []

    for symbol in all_symbols:
        try:
            trades = client.get_my_trades(symbol=symbol)
            for trade in trades:
                trade_info = {
                    'datetime': datetime.fromtimestamp(trade['time'] / 1000),
                    'type': 'buy' if trade['isBuyer'] else 'sell',
                    'symbol': trade['symbol'],
                    'quantity': float(trade['qty']),
                    'average_cost': float(trade['price']),
                    'amount': float(trade['qty']) * float(trade['price']),
                    'commission': float(trade['commission']),
                }
                transactions.append(trade_info)
        except Exception as e:
            print(f"Could not fetch trades for symbol {symbol}: {e}")

    return transactions

fetch_all_transactions(client)

[{'datetime': datetime.datetime(2024, 9, 10, 20, 20, 49, 898000),
  'type': 'buy',
  'symbol': 'BTCUSDT',
  'quantity': 0.0066,
  'average_cost': 56928.01,
  'amount': 375.724866,
  'commission': 0.0},
 {'datetime': datetime.datetime(2024, 9, 10, 20, 20, 49, 898000),
  'type': 'buy',
  'symbol': 'BTCUSDT',
  'quantity': 0.00352,
  'average_cost': 56934.05,
  'amount': 200.407856,
  'commission': 0.0},
 {'datetime': datetime.datetime(2024, 9, 10, 20, 20, 49, 898000),
  'type': 'buy',
  'symbol': 'BTCUSDT',
  'quantity': 0.00782,
  'average_cost': 56934.07,
  'amount': 445.2244274,
  'commission': 0.0},
 {'datetime': datetime.datetime(2024, 9, 10, 20, 20, 49, 898000),
  'type': 'buy',
  'symbol': 'BTCUSDT',
  'quantity': 0.00475,
  'average_cost': 56934.18,
  'amount': 270.43735499999997,
  'commission': 0.0},
 {'datetime': datetime.datetime(2024, 9, 10, 20, 20, 49, 898000),
  'type': 'buy',
  'symbol': 'BTCUSDT',
  'quantity': 0.00817,
  'average_cost': 56938.0,
  'amount': 465.18346,
 

In [9]:
df = fetch_all_transactions(client)

In [11]:
df[1]['average_cost']

56934.05

In [2]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Example content formatted into sections
content = {
    "Header": "Nitin Yadav\nny78455@gmail.com | 7505616125\nGitHub: github.com/ny78455 | LinkedIn: linkedin.com/in/nitin-yadav",
    "Skills": "Languages:\nC/C++, Java, Python, JavaScript, R, SQL\n\n"
              "Technologies & Tools:\nMicrosoft Azure, NLTK, Spacy, Pandas, NumPy, Matplotlib, TensorFlow, PyTorch, Scikit-learn,\n"
              "Keras, Plotly, Tableau, Hadoop, Hive, AWS, MySQL, MongoDB, Docker, CI/CD, MLflow, Flask, FastAPI, APScheduler, Git,\n"
              "GitHub, Hugging Face, VS Code, PyCharm, Pandas-TA, VAR Models, Binance API, Google Gemini API, Beautiful Soup, Selenium",
    "Work Experience": "Vantrade Investments, Bhopal\nFounder & CEO\nApril 2024 - Present\n\n"
                       "- Pioneered the application of generative AI in trading, achieving an exceptional 80-90% win rate\n"
                       "with a maximum drawdown of 5-6%.\n"
                       "- Automated trade execution based on dynamic alpha signals, leading to a 30-40% ROI within 15-20\n"
                       "days.\n"
                       "- Developed an AI-powered financial advisor trained on over 3,500 books using Google Gemini embeddings, T5-tokenizer, and BERT models.\n"
                       "- Generated comprehensive market insights reports using advanced NLP and web scraping\n"
                       "techniques, analyzing over 1M data points to deliver actionable recommendations.\n"
                       "- Engineered an automated deployment pipeline on Microsoft Azure with MLflow, reducing manual\n"
                       "processes by 15 hours per week for the data science team.\n\n"
                       "Grip Invest, Remote\nQuant Analyst\nApril 2024 - June 2024\n\n"
                       "- Developed dynamic trading strategies and alpha models, resulting in 2-3% returns over short\n"
                       "periods (1-2 days) with a win rate of 40-60%.\n"
                       "- Implemented and deployed alphas on AWS, leveraging CI/CD pipelines and MLflow for seamless\n"
                       "model management.\n"
                       "- Technologies: Java, Python, AWS Step Functions, AWS Batch, Pandas, Pandas-TA, Indicators,\n"
                       "Dill, TensorFlow, CI/CD, MLflow.\n\n"
                       "IIIT Bhopal\nResearch Intern\nJune 2023 - Sept 2023\n\n"
                       "- Collected and processed a dataset of over 10,000 images of infected potato leaves using data\n"
                       "collection tools, image augmentation, magnification, and feature extraction techniques.\n"
                       "- Trained an ensemble deep learning model comprising CNN, GNN, and RL, achieving 98%\n"
                       "accuracy.",
    "Education": "Indian Institute of Information Technology Bhopal\nB.Tech in Electronics and Communication Engineering\nCGPA: 7.06/10",
    "Project Work": "Integrated Quality Monitoring Discrepancy System (2023):\n"
                    "Developed a web application to detect discrepancies in road quality monitoring using AI/ML and\n"
                    "Data Science for the Ministry of Rural Development. Utilized TensorFlow, Keras, React, Hadoop,\n"
                    "Scikit-learn, and MySQL.\n\n"
                    "Trade/Sync (2024):\n"
                    "Engineered multiple alpha models and trading strategies to generate master signals for autonomous\n"
                    "buy/sell orders based on market conditions, achieving an 80%+ win rate with a 6-8% drawdown.\n"
                    "Added market insights features for deep analysis. Technologies: PyTorch, HTML, CSS, React,\n"
                    "MongoDB, Pandas-TA, Plotly.\n\n"
                    "Personalized Retail Chatbot (2024):\n"
                    "Created a personalized chatbot for Walmart, providing real-time product recommendations and\n"
                    "relevant suggestions based on live data, with integrated AR/VR experiences and virtual try-ons.",
    "Awards and Certificates": "Winner of HumanAlze Fintech Hackathon (2024):\nSecured the top position among 50,000 participants in the HAFH.\n\n"
                               "Top 3 in Smart India Hackathon (2023):\nRanked third among 200,000 participants in SIH 2023.\n\n"
                               "Finalist in Scale+91 Hackathon (2024):\nAchieved top 10 ranking among 100,000 participants and presented ideas at the Fintech Festival\nIndia (FFI)."
}

# Create a PDF document
with PdfPages('styled_resume.pdf') as pdf:
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(8.5, 11))
    ax.axis('off')  # Hide axes

    # Define styles
    title_style = {'fontsize': 24, 'fontweight': 'bold', 'color': '#333333'}
    header_style = {'fontsize': 16, 'fontweight': 'bold', 'color': '#1F77B4'}
    text_style = {'fontsize': 12, 'color': '#555555'}
    date_style = {'fontsize': 12, 'color': '#1F77B4', 'fontstyle': 'italic'}

    # Add Header
    ax.text(0.1, 0.95, content["Header"], **title_style)

    # Add a line
    ax.plot([0.1, 0.9], [0.93, 0.93], color='black', lw=0.5)

    # Add sections
    y_position = 0.9
    for section, text in content.items():
        if section == "Header":
            continue
        ax.text(0.1, y_position, section, **header_style)
        y_position -= 0.03
        if section in ["Work Experience", "Education"]:
            text_lines = text.split('\n')
            for line in text_lines:
                if "20" in line:  # Assuming dates contain "20" (you can adjust this)
                    ax.text(0.1, y_position, line, **date_style)
                else:
                    ax.text(0.1, y_position, line, **text_style)
                y_position -= 0.03
        else:
            ax.text(0.1, y_position, text, **text_style)
            y_position -= 0.06

        # Add a line after each section
        ax.plot([0.1, 0.9], [y_position, y_position], color='black', lw=0.5)
        y_position -= 0.03

    # Save the page
    pdf.savefig(fig)
    plt.close()

print("Styled resume saved as styled_resume.pdf")


Styled resume saved as styled_resume.pdf


In [1]:
from transformers import GPT2Tokenizer, GPT2Model

# Load pre-trained GPT model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

def encode_query(text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

encoded_query = encode_query("I want to buy chips")


/home/nitin/Downloads/Project/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
predefined_parameters = {
    "item": ['Hummus, Dips, & Salsa', 'Energy Drinks', 'Oils & Shortening',
       'Fresh Soups & Salads', 'Game Time Faves', 'Sparkling Water',
       'Better for you', 'Healthy Snacks', 'Pudding & Gelatin',
       'Pretzels', 'Granola Bars', 'Baking Nuts & Seeds', 'Yeast', 'Milk',
       'Cheese', 'Halloween candy', 'Canned & Powdered Milks',
       'Specialty Cheeses & Meats', 'Bacon, Hot Dogs, Sausage',
       'Variety Pack Snacks', 'Drink Mixes', 'Eggs',
       'Orange Juice & Chilled', 'Cream & Creamers', 'Beverage Deals',
       'Hard candy & lollipops', 'Meat Sticks', 'Fruit Snacks',
       'Chocolate', 'Fresh Pizza', 'Sugars & Sweeteners', 'New Arrivals',
       'Biscuits, Cookies, Doughs', 'Top baking brands', 'Mints',
       'Prepared Meals & Sides', 'Yogurt', 'Popcorn', 'Chips',
       'Rotisserie Chicken', 'Juices', 'Deli Meat & Cheese',
       'Gummy & chewy candy', 'Non-Alcoholic Mixers',
       'Butter & Margarine', 'Sports Drinks', 'Multipacks & bags',
       'Crackers', 'Fresh Pasta', 'Flours & Meals', 'Easy to make',
       'Snack Nuts', 'Snacks & Fresh Sandwiches', 'Great Value Beverages',
       'Sour Cream & Chilled Dips', "Kids' Multi-Packs", 'Gum',
       'Beef Jerky', 'Baking Soda & Starch', 'Tea', 'Fresh Bakery Breads',
       'Breakfast Meats', 'Breakfast Breads', 'Cookies & Brownies',
       'Grilling', 'Fresh Food', 'Rolls & Buns', 'Cakes & Cupcakes',
       'Emergency & Institutional food', 'Cereal & Granola',
       'Dairy & Eggs', 'Breakfast Beverages', 'Coffee Accessories',
       'Tortillas', 'Pies', 'Donuts, Muffins & Pastries', 'Roast Type',
       'Muffins & Pastries', 'Sweet Treats', 'Sliced Bread',
       'Pancakes, Waffles & Syrups', 'Herbs, spices & seasonings',
       'Frozen Meals & Snacks', 'Fresh Vegetables', 'Salad Kits & Bowls',
       'Frozen Potatoes', 'Organic Produce', 'Fresh Dressings',
       'Condiments', 'Salsa & Dips', 'Cooking oils & vinegars',
       'Plant-based Protein & Tofu', 'Soup', 'Fresh Fruits',
       'Frozen Meat & Seafood', 'Frozen Desserts', 'Canned goods',
       'Frozen Breakfast', 'Cut Fruits & Vegetables', 'Pasta & pizza',
       'International foods', 'Coffee Additives', 'Fresh Herbs',
       'Frozen Pizza, Pasta, & Breads', 'Rice, grains & dried beans',
       'Frozen Produce', 'Coffee By Type', 'Canned vegetables',
       'Hot Cereals', 'Packaged meals & side dishes', 'Fresh Flowers',
       'Wine', 'Spirits', 'Beer'],
    "category": ['Deli/Hummus, Dips, & Salsa', 'Beverages/Energy Drinks',
       'Baking/Oils & Shortening', 'Deli/Fresh Soups & Salads',
       'Meat & Seafood/Game Time Faves', 'Beverages/Sparkling Water',
       'Candy/Better for you', 'Snacks/Healthy Snacks',
       'Dairy & Eggs/Pudding & Gelatin', 'Snacks/Pretzels',
       'Snacks/Granola Bars', 'Baking/Baking Nuts & Seeds',
       'Baking/Yeast', 'Dairy & Eggs/Milk', 'Dairy & Eggs/Cheese',
       'Candy/Halloween candy', 'Baking/Canned & Powdered Milks',
       'Deli/Specialty Cheeses & Meats', 'Deli/Bacon, Hot Dogs, Sausage',
       'Snacks/Variety Pack Snacks', 'Beverages/Drink Mixes',
       'Dairy & Eggs/Eggs', 'Beverages/Orange Juice & Chilled',
       'Dairy & Eggs/Cream & Creamers', 'Beverages/Beverage Deals',
       'Candy/Hard candy & lollipops', 'Snacks/Meat Sticks',
       'Snacks/Fruit Snacks', 'Candy/Chocolate', 'Deli/Fresh Pizza',
       'Baking/Sugars & Sweeteners', 'Beverages/New Arrivals',
       'Dairy & Eggs/Biscuits, Cookies, Doughs',
       'Baking/Top baking brands', 'Candy/Mints',
       'Deli/Prepared Meals & Sides', 'Dairy & Eggs/Yogurt',
       'Snacks/Popcorn', 'Snacks/Chips', 'Deli/Rotisserie Chicken',
       'Beverages/Juices', 'Deli/Deli Meat & Cheese',
       'Candy/Gummy & chewy candy', 'Beverages/Non-Alcoholic Mixers',
       'Dairy & Eggs/Butter & Margarine', 'Beverages/Sports Drinks',
       'Candy/Multipacks & bags', 'Snacks/Crackers', 'Deli/Fresh Pasta',
       'Baking/Flours & Meals', 'Baking/Easy to make',
       'Snacks/Snack Nuts', 'Deli/Snacks & Fresh Sandwiches',
       'Beverages/Great Value Beverages',
       'Dairy & Eggs/Sour Cream & Chilled Dips',
       "Beverages/Kids' Multi-Packs", 'Candy/Gum', 'Snacks/Beef Jerky',
       'Baking/Baking Soda & Starch', 'Beverages/Tea',
       'Bakery & Bread/Fresh Bakery Breads',
       'Breakfast & Cereal/Breakfast Meats',
       'Breakfast & Cereal/Breakfast Breads',
       'Bakery & Bread/Breakfast Breads',
       'Bakery & Bread/Cookies & Brownies', 'Meat & Seafood/Grilling',
       'Meat & Seafood/Fresh Food', 'Bakery & Bread/Rolls & Buns',
       'Bakery & Bread/Cakes & Cupcakes',
       'Pantry/Emergency & Institutional food',
       'Breakfast & Cereal/Cereal & Granola',
       'Breakfast & Cereal/Dairy & Eggs',
       'Breakfast & Cereal/Breakfast Beverages',
       'Coffee/Coffee Accessories', 'Bakery & Bread/Tortillas',
       'Bakery & Bread/Pies', 'Bakery & Bread/Donuts, Muffins & Pastries',
       'Coffee/Roast Type', 'Breakfast & Cereal/Muffins & Pastries',
       'Bakery & Bread/Sweet Treats', 'Bakery & Bread/Sliced Bread',
       'Breakfast & Cereal/Pancakes, Waffles & Syrups',
       'Pantry/Herbs, spices & seasonings',
       'Frozen/Frozen Meals & Snacks', 'Fresh Produce/Fresh Vegetables',
       'Fresh Produce/Salad Kits & Bowls', 'Frozen/Frozen Potatoes',
       'Fresh Produce/Organic Produce', 'Fresh Produce/Fresh Dressings',
       'Pantry/Condiments', 'Fresh Produce/Salsa & Dips',
       'Pantry/Cooking oils & vinegars',
       'Fresh Produce/Plant-based Protein & Tofu', 'Pantry/Soup',
       'Fresh Produce/Fresh Fruits', 'Frozen/Frozen Meat & Seafood',
       'Frozen/Frozen Desserts', 'Pantry/Canned goods',
       'Breakfast & Cereal/Frozen Breakfast',
       'Fresh Produce/Cut Fruits & Vegetables', 'Pantry/Pasta & pizza',
       'Pantry/International foods', 'Coffee/Coffee Additives',
       'Fresh Produce/Fresh Herbs',
       'Frozen/Frozen Pizza, Pasta, & Breads',
       'Pantry/Rice, grains & dried beans', 'Frozen/Frozen Produce',
       'Coffee/Coffee By Type', 'Pantry/Canned vegetables',
       'Breakfast & Cereal/Hot Cereals',
       'Pantry/Packaged meals & side dishes', 'Frozen/Frozen Breakfast',
       'Fresh Produce/Fresh Flowers', 'Alcohol/Wine', 'Alcohol/Spirits',
       'Alcohol/Beer'],
    "category": ['AriZona', 'Artisan Breads', 'Bacon', 'Bagels',
       'Baguettes, Loaves & Breadsticks', 'Bakery Cookies', 'BBQ Ribs',
       'Beef & Lamb', 'Bigelow', 'Biscuits, Doughs & Rolls',
       'Brioche Rolls & Buns', 'Brownies & Dessert Bars', 'Burgers',
       'Cake and cupcakes', 'Cake Decorations', 'Camping food',
       'Celestial Seasonings', 'Cereal Bars', 'Cheese', 'Cheesecakes',
       'Chicken', 'Cocoa', 'Coffee', 'Coffee Filters', 'Coffee Makers',
       'Coffee Mugs', 'Corn Tortillas', 'Cream Cheese', 'Creme Pies',
       'Croissants', 'Croissants ', 'Cupcakes & Cake Bites',
       'Danishes, Sweet Rolls & Turnovers', 'Dark', 'Deli meat & cheese',
       'Dessert & Celebration Cakes', 'Dinner Rolls', 'Donuts', 'Eggs',
       'Emergency food', 'English Muffins', 'Flatbreads',
       'Flour Tortillas', 'French & Italian Loaves', 'Fresh Baked Donuts',
       'Fresh bakery breads', 'Fresh Fruits', 'Fresh vegetables',
       'Fruit Pies', 'Gold Peak', 'Granola', 'Great Value',
       'Grilled Mexican Corn', 'Grilled Pork Tenderloin',
       'Grilled Sausage Dogs', 'Grilled Shrimp Kabobs',
       'Grilled Stuffed Chicken', 'Hamburger Buns', 'Hot Dog Buns',
       'Ice Cream Cakes', 'Institutional food', 'Italian Bread', 'Juice',
       'Light', 'Lipton', 'Medium', 'Muffins', 'Muffins & Cakes',
       'Muffins & Mini Muffins', 'Muffins and Donuts',
       'Muffins, donuts, pastries', 'Multigrain Bread',
       'Pancake & Waffle Mixes', 'Pastries', 'Pitas & Wraps',
       'Plant-base alternatives', 'Pork', 'Prepared meals & sides',
       'Pukka Tea', 'Pumpkin, Pecan & Custard Pies', 'Pure Leaf',
       'Red Rose', 'Rollbacks', 'Rolls', 'Rye & Sourdough Bread',
       'Salad kits and bowls', 'Sandwich Rolls', 'Seafood', 'Shop all',
       'Shop All', 'Single Serve Desserts', 'Smoothies',
       'Snack Mixes & Treats', 'Snapple', 'Specialty cheeses & meats',
       'Sweet Treats', 'Syrups', 'Tazo', 'Tea', 'Teavana', 'Toast',
       'Tortillas, Pitas & Wraps', 'Traditional Medicinals', 'Twinings',
       'White Bread', 'Whole Wheat Bread', 'Yogi', 'Yogurt', 'Blush Wine',
       'Boxed Wine', 'Brandy & Cognac', 'Cocktails', 'Domestic Beer',
       'Flavored Beer & Hard Seltzers', 'Gin', 'Imported Beer',
       'Liqueurs & Cordials', 'Red Wine', 'Rum', 'Sparkling Wine',
       'Specialty Wine', 'Tequila', 'Vodka', 'Whiskey', 'White Wine']
}


In [3]:
from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode predefined parameters
encoded_parameters = {key: model.encode(values) for key, values in predefined_parameters.items()}


2024-08-11 12:47:37.323941: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 12:47:37.453818: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 12:47:37.488518: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-11 12:47:37.682824: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-11 12:47:39.725795: W tensorflow/compiler/tf2

In [4]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def extract_parameters_from_embedding(query_embedding, encoded_parameters):
    extracted_parameters = {}
    for key, embeddings in encoded_parameters.items():
        # Calculate cosine similarity
        similarities = cosine_similarity([query_embedding], embeddings)[0]
        # Find the best match
        best_match_index = np.argmax(similarities)
        extracted_parameters[key] = predefined_parameters[key][best_match_index]
    return extracted_parameters

# Example usage
query = "I want to buy chips"
query_embedding = model.encode(query)
extracted_params = extract_parameters_from_embedding(query_embedding, encoded_parameters)
print(extracted_params['item'])
print(extracted_params)


Chips
{'item': 'Chips', 'category': 'Bakery Cookies'}


In [5]:
import requests
from bs4 import BeautifulSoup

# Base URL
base_url = "https://www.walmart.com/search"

# Parameters to append to the base URL

q =  extracted_params['item']
sort = "best_match"
catId =  ""
max_price = ""
brand = ""
exclude_oos = ""
category = ""
global_product_type = ""
nutrient_content_claims = ""
food_condition = ""

# Generate the full URL with parameters

full_url = f"https://www.walmart.com/search?q={q}&sort={sort}&max_price={max_price}&facet=brand%3A{brand}%7C%7Cexclude_oos%3A{exclude_oos}%7C%7Ccategory%3A{category}%7C%7Cglobal_product_type%3A{global_product_type}%7C%7Cnutrient_content_claims%3A{nutrient_content_claims}%7C%7Cfood_condition%3A{food_condition}"
# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Send a GET request to the full URL
response = requests.get(full_url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Remove advertisement tags (if any)
    for ad in soup.select("div[class*='ph1 pa0-xl bb b--near-white w-25']"):
        ad.decompose()

    # Updated selector for product titles
    product_titles = soup.select("div[class*='mb0 ph0-xl pt0-xl bb b--near-white w-25 pb3-m ph1'], div[class*='mb0 ph0-xl pt0-xl w-25 pb3-m ph1']")

    if len(product_titles) == 0:
        print("No product titles found. The structure of the page might have changed.")
    else:
        # Print the product titles in groups of 5
        for i in range(0, len(product_titles), 5):
            group = product_titles[i:i+5]
            print("Group of 5 products:")
            for title in group:
                print(f"- {title.get_text(strip=True)}")
            print()  # Add a blank line between groups
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Group of 5 products:
- Frito-Lay Classic Snack Mix Variety Pack Snack Chips, 42 Count MultipackBest sellerOptionsSponsored$1998current price $19.9851.2 ¢/ozFrito-Lay Classic Snack Mix Variety Pack Snack Chips, 42 Count Multipack2412.8 out of 5 Stars. 241 reviewsEBT eligibleSave withPickuptodayDeliverytodayShipping, arrivestodaySubscribe
- Frito-Lay Bold Mix Variety Pack Snack Chips, 28 Count BagsOptionsSponsored$1598current price $15.9858.6 ¢/ozFrito-Lay Bold Mix Variety Pack Snack Chips, 28 Count Bags602.5 out of 5 Stars. 60 reviewsEBT eligibleSave withPickuptodayDeliverytodayShipping, arrivestodaySubscribe
- Doritos Nacho Cheese Flavor Tortilla Snack Chips, 1oz Bags, 10 Count MultipackBest sellerAddSponsoredNow$548current price Now $5.48$6.82Was $6.8254.8 ¢/ozDoritos Nacho Cheese Flavor Tortilla Snack Chips, 1oz Bags, 10 Count Multipack1123.2 out of 5 Stars. 112 reviewsEBT eligibleSave withPickuptodayDeliverytodayShipping, arrivestodaySubscribe
- Lay's Classic Potato Snack Chips, 1 o

In [6]:
import pandas as pd
df = pd.read_csv('WMT_Grocery_202209.csv')

/tmp/ipykernel_1307708/1747429656.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('WMT_Grocery_202209.csv')


In [7]:
df

,index,SHIPPING_LOCATION,DEPARTMENT,CATEGORY,SUBCATEGORY,BREADCRUMBS,SKU,PRODUCT_URL,PRODUCT_NAME,BRAND,PRICE_RETAIL,PRICE_CURRENT,PRODUCT_SIZE,PROMOTION,RunDate,tid
0,0,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",110895339,https://www.walmart.com/ip/Marketside-Roasted-...,"Marketside Roasted Red Pepper Hummus, 10 Oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163804
1,1,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",105455228,https://www.walmart.com/ip/Marketside-Roasted-...,"Marketside Roasted Garlic Hummus, 10 Oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163805
2,2,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",128642379,https://www.walmart.com/ip/Marketside-Classic-...,"Marketside Classic Hummus, 10 Oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163806
3,3,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",366126367,https://www.walmart.com/ip/Marketside-Everythi...,"Marketside Everything Hummus, 10 oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163807
4,4,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",160090316,https://www.walmart.com/ip/Price-s-Jalapeno-Di...,"Price's Jalapeno Dip, 12 Oz.",Price's,3.12,3.12,12,NaN,2022-09-11 21:20:04,16163808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568529,568529,70072,Alcohol,Wine,White Wine,Alcohol/Wine,593600139,https://www.walmart.com/ip/Farm-Fresh-Blueberr...,Farm Fresh Blueberry Moscato 750ml,Farm Fresh Wine Company,9.98,9.98,750,NaN,2022-09-11 21:20:04,16732333
568530,568530,70072,Alcohol,Wine,White Wine,Alcohol/Wine,333403243,https://www.walmart.com/ip/Farm-Fresh-Peach-Mo...,Farm Fresh Peach Moscato 750 Ml,Farm Fresh Wine Company,9.98,9.98,750,NaN,2022-09-11 21:20:04,16732334
568531,568531,70072,Alcohol,Wine,White Wine,Alcohol/Wine,526588325,https://www.walmart.com/ip/Farm-Fresh-Raspberr...,Farm Fresh Raspberry Moscato 750ml,Farm Fresh Wine Company,9.98,9.98,750,NaN,2022-09-11 21:20:04,16732335
568532,568532,70072,Alcohol,Wine,White Wine,Alcohol/Wine,286992782,https://www.walmart.com/ip/Farm-Fresh-Mango-Mo...,Farm Fresh Mango Moscato 750ml,Farm Fresh Wine Company,9.98,9.98,750,NaN,2022-09-11 21:20:04,16732336


In [8]:
df['SUBCATEGORY'].unique()

array([nan, 'AriZona', 'Artisan Breads', 'Bacon', 'Bagels',
       'Baguettes, Loaves & Breadsticks', 'Bakery Cookies', 'BBQ Ribs',
       'Beef & Lamb', 'Bigelow', 'Biscuits, Doughs & Rolls',
       'Brioche Rolls & Buns', 'Brownies & Dessert Bars', 'Burgers',
       'Cake and cupcakes', 'Cake Decorations', 'Camping food',
       'Celestial Seasonings', 'Cereal Bars', 'Cheese', 'Cheesecakes',
       'Chicken', 'Cocoa', 'Coffee', 'Coffee Filters', 'Coffee Makers',
       'Coffee Mugs', 'Corn Tortillas', 'Cream Cheese', 'Creme Pies',
       'Croissants', 'Croissants ', 'Cupcakes & Cake Bites',
       'Danishes, Sweet Rolls & Turnovers', 'Dark', 'Deli meat & cheese',
       'Dessert & Celebration Cakes', 'Dinner Rolls', 'Donuts', 'Eggs',
       'Emergency food', 'English Muffins', 'Flatbreads',
       'Flour Tortillas', 'French & Italian Loaves', 'Fresh Baked Donuts',
       'Fresh bakery breads', 'Fresh Fruits', 'Fresh vegetables',
       'Fruit Pies', 'Gold Peak', 'Granola', 'Great Val

In [4]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_email(recipient, team_name):
    # Your email credentials
    sender_email = "investmentsvantrade@gmail.com"
    sender_password = "yadav1234"
    smtp_server = "smtp.gmail.com"  # E.g., 'smtp.gmail.com' for Gmail
    smtp_port = 587  # Typically 587 for TLS, 465 for SSL
    
    # Creating the message
    subject = f"Funding Proposal for High-Performance Trading Strategies and Web Application"
    body = f"""
    Team {team_name},

    I hope this message finds you well.

    I am writing to introduce a trading platform I have developed, which integrates multiple advanced strategies and features that have consistently delivered exceptional returns. Over the past 20-30 days, these strategies have achieved returns of up to 110%, with zero losses recorded during this period.

    This platform is more than just a collection of strategies; it is a comprehensive web application that includes all essential trading features, designed to provide a seamless and profitable experience for users. The robustness of the strategies and the sophisticated design of the platform offer a unique opportunity for substantial gains in the competitive trading landscape.

    To give you a better understanding of the platform's capabilities, I have prepared a short demo video, which you can view here.

    Given the impressive performance and the potential for further scalability, I am seeking funding to enhance and expand the platform. I believe that collaboration with a leading high-frequency trading firm such as {team_name} would not only accelerate growth but also align with your firm's strategic objectives in leveraging cutting-edge trading technology.

    I would welcome the opportunity to discuss this in more detail and explore potential synergies. Please let me know a convenient time for you, and I will be happy to arrange a meeting.

    Thank you for considering this opportunity. I look forward to the possibility of working together.

    Best regards,
    Nitin Yadav
    """

    # Setting up the MIME
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient
    msg['Subject'] = subject

    # Attaching the body with the msg instance
    msg.attach(MIMEText(body, 'plain'))

    # Sending the email
    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Enable security
        server.login(sender_email, sender_password)
        text = msg.as_string()
        server.sendmail(sender_email, recipient, text)
        server.quit()

        print(f"Email successfully sent to {recipient}")
    except Exception as e:
        print(f"Failed to send email to {recipient}. Error: {str(e)}")

if __name__ == "__main__":
    # List of recipient emails and their corresponding team/company names
    recipients = [
        {"email": "mohitverma.exe@gmail.com", "team": "Mohit"},
        {"email": "ny78455@gmail.com", "team": "Nitin"},
    ]

    # Sending emails
    for recipient_info in recipients:
        send_email(recipient_info['email'], recipient_info['team'])


Failed to send email to mohitverma.exe@gmail.com. Error: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 98e67ed59e1d1-2d3e307640fsm3555701a91.28 - gsmtp')
Failed to send email to nym78455@gmail.com. Error: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 98e67ed59e1d1-2d3ac7f21d2sm7298637a91.28 - gsmtp')


In [ ]:
import yagmail

# Only needed once, saves credentials to keyring
yagmail.register('your_email@gmail.com', 'your_oauth2_token')


In [ ]:
import yagmail

def send_email(recipient, team_name):
    # Initialize the yagmail.SMTP object
    yag = yagmail.SMTP('your_email@gmail.com')

    subject = "Funding Proposal for High-Performance Trading Strategies and Web Application"
    body = f"""
    Team {team_name},

    I hope this message finds you well.

    I am writing to introduce a trading platform I have developed, which integrates multiple advanced strategies and features that have consistently delivered exceptional returns. Over the past 20-30 days, these strategies have achieved returns of up to 110%, with zero losses recorded during this period.

    This platform is more than just a collection of strategies; it is a comprehensive web application that includes all essential trading features, designed to provide a seamless and profitable experience for users. The robustness of the strategies and the sophisticated design of the platform offer a unique opportunity for substantial gains in the competitive trading landscape.

    To give you a better understanding of the platform's capabilities, I have prepared a short demo video, which you can view here.

    Given the impressive performance and the potential for further scalability, I am seeking funding to enhance and expand the platform. I believe that collaboration with a leading high-frequency trading firm such as {team_name} would not only accelerate growth but also align with your firm's strategic objectives in leveraging cutting-edge trading technology.

    I would welcome the opportunity to discuss this in more detail and explore potential synergies. Please let me know a convenient time for you, and I will be happy to arrange a meeting.

    Thank you for considering this opportunity. I look forward to the possibility of working together.

    Best regards,
    Nitin Yadav
    """

    # Send the email
    try:
        yag.send(
            to=recipient,
            subject=subject,
            contents=body
        )
        print(f"Email successfully sent to {recipient}")
    except Exception as e:
        print(f"Failed to send email to {recipient}. Error: {str(e)}")

if __name__ == "__main__":
    # List of recipient emails and their corresponding team/company names
    recipients = [
        {"email": "mohitverma.exe@gmail.com", "team": "Mohit"},
        {"email": "ny78455@gmail.com", "team": "Nitin"},
    ]

    # Sending emails
    for recipient_info in recipients:
        send_email(recipient_info['email'], recipient_info['team'])


In [5]:
import os
import pickle
import google.auth.transport.requests
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# Path to your client_secret.json file
CLIENT_SECRET_FILE = 'client_secret_228285637763-h12qkiqhg0jqhd7q5v9u6buedr9bte2l.apps.googleusercontent.com.json'
SCOPES = ['https://mail.google.com/']

def get_oauth2_token():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds.token

token = get_oauth2_token()
print("Your OAuth2 token is:", token)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=228285637763-h12qkiqhg0jqhd7q5v9u6buedr9bte2l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A39757%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=83kNYhdMrbl1AVNzE3bT0mQ99HQft7&access_type=offline


KeyboardInterrupt: 

In [14]:
import pandas as pd
import numpy as np

# Load the CSV file into a DataFrame
df = pd.read_csv('check1.csv', parse_dates=['Datetime'])

# Initialize backtest parameters
initial_balance = 10000  # Starting capital
balance = initial_balance
position_size = 100  # The size of the option contract (e.g., 100 units)
portfolio = []
returns = []
trades = []  # To store trade details

# Option pricing parameters
option_multiplier = 0.1  # Assuming options are priced at 10% of the stock price for simplicity

# Iterate over the DataFrame to execute the strategy
for i in range(len(df)):
    row = df.iloc[i]
    prediction = row['Prediction']
    
    # Handle option purchase based on prediction
    if prediction > 0:
        # Determine the option's strike price (use the Close price)
        strike_price = row['Close']
        option_price = strike_price * option_multiplier
        
        # Check if balance allows purchase
        if balance >= option_price * position_size:
            # Buy the option (call option in this case)
            balance -= option_price * position_size
            portfolio.append({
                'buy_price': option_price,
                'strike_price': strike_price,
                'expiry': i + prediction,  # Option expires after 'prediction' candlesticks
                'direction': 'call'
            })
            trades.append({
                'Datetime': row['Datetime'],
                'Type': 'Buy',
                'Strike Price': strike_price,
                'Option Price': option_price,
                'Balance After Trade': balance
            })

    # Check portfolio for expired options
    for position in portfolio.copy():
        if i == position['expiry']:
            # Determine the closing price at expiry
            close_price = df.iloc[i]['Close']
            option_value = max(0, close_price - position['strike_price']) * option_multiplier
            
            # Sell the option and calculate the profit/loss
            balance += option_value * position_size
            profit_loss = (option_value - position['buy_price']) * position_size
            returns.append(profit_loss)
            trades.append({
                'Datetime': df.iloc[i]['Datetime'],
                'Type': 'Sell',
                'Strike Price': position['strike_price'],
                'Option Price': option_value,
                'Profit/Loss': profit_loss,
                'Balance After Trade': balance
            })
            portfolio.remove(position)

# Calculate the final portfolio value and performance metrics
total_returns = balance - initial_balance

# Convert trades to a DataFrame for easier viewing
trades_df = pd.DataFrame(trades)

# Display the trades and summary
print("All Trades:")
print(trades_df)
print("\nSummary:")
print(f"Initial Balance: {initial_balance}")
print(f"Final Balance: {balance}")
print(f"Total Returns: {total_returns}")
print(f"Total Return Percentage: {total_returns / initial_balance * 100}%")


All Trades:
Empty DataFrame
Columns: []
Index: []

Summary:
Initial Balance: 10000
Final Balance: 10000
Total Returns: 0
Total Return Percentage: 0.0%


In [2]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('check1.csv', parse_dates=['Datetime'])

# Initialize backtest parameters
initial_balance = 100000  # Starting capital
balance = initial_balance
position_size = 1  # Number of futures contracts per trade
portfolio = []
returns = []

# Iterate over the DataFrame to execute the strategy
for i in range(len(df)):
    row = df.iloc[i]
    prediction = row['Prediction']
    
    # Enter a long position based on the prediction
    if prediction > 0:
        entry_price = row['Close']
        expiry_index = i + prediction
        
        # Ensure expiry_index doesn't exceed DataFrame length
        if expiry_index < len(df):
            exit_price = df.iloc[expiry_index]['Close']
            
            # Calculate profit/loss for the futures position
            profit_loss = (exit_price - entry_price) * position_size
            
            # Update balance and record the trade
            balance += profit_loss
            returns.append(profit_loss)

# Calculate the final portfolio value and performance metrics
total_returns = balance - initial_balance
print(f"Initial Balance: {initial_balance}")
print(f"Final Balance: {balance}")
print(f"Total Returns: {total_returns}")
print(f"Total Return Percentage: {total_returns / initial_balance * 100}%")


Initial Balance: 100000
Final Balance: 235302.43000000023
Total Returns: 135302.43000000023
Total Return Percentage: 135.30243000000021%


In [28]:
close_price = 64291
prediction = 1

target_sell_price = close_price + ((prediction + 1) * ((close_price / 100) * 0.124))

In [4]:
from fpdf import FPDF

# Initialize a PDF document
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

# Title
pdf.set_font("Arial", 'B', 16)
pdf.cell(200, 10, txt="Comprehensive Contract Farming Platform", ln=True, align='C')
pdf.ln(10)

# Section 1: Detailed Explanation of the Proposed Solution
pdf.set_font("Arial", 'B', 12)
pdf.cell(200, 10, txt="1. Detailed Explanation of the Proposed Solution", ln=True)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 10, txt="""The proposed platform is designed to create a holistic ecosystem that supports contract farming. The key components of the solution are as follows:

1.1 **Middlemen Role**:
- Government or private organizations will act as intermediaries, purchasing raw materials from farmers under contract.
- These middlemen will sell the processed or raw products at a target price, aiming for a 40% profit margin above the purchase price.

1.2 **Weather Forecasting and Agricultural Tools**:
- The platform will provide farmers with access to weather forecasting tools and other essential resources for effective farm management.
- Additional tools include soil analysis, crop disease alerts, and yield prediction models.

1.3 **Equipment Rental Service**:
- The platform will facilitate the renting of farming equipment from nearby villages.
- Farmers can rent tractors, loaders, and other machinery at affordable rates, improving access to modern equipment.

1.4 **Automation in Irrigation**:
- IoT-based automation will be integrated into the platform to automate irrigation processes.
- Sensors and smart irrigation systems will optimize water usage, reducing labor costs and enhancing efficiency.""")

# Section 2: How It Addresses the Problem
pdf.set_font("Arial", 'B', 12)
pdf.cell(200, 10, txt="2. How It Addresses the Problem", ln=True)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 10, txt="""The proposed platform tackles the core issues faced by farmers, particularly income instability and market uncertainties. The solution addresses these challenges through:

2.1 **Guaranteed Buyers and Stable Pricing**:
- By connecting farmers with buyers through contractual agreements, the platform ensures that farmers have a secure market for their produce.
- The involvement of middlemen ensures that farmers receive a fair and consistent price, mitigating the risks of market fluctuations.

2.2 **Access to Critical Farming Tools**:
- The platform provides farmers with essential tools, such as weather forecasts and soil analysis, enabling them to make informed decisions and optimize crop yields.

2.3 **Reduction of Operational Costs**:
- The equipment rental service and automated irrigation systems reduce the need for large capital investments and labor, lowering overall operational costs.

2.4 **Enhanced Productivity and Profitability**:
- The combination of secure contracts, modern equipment, and advanced farming tools enhances the productivity of farmers, leading to increased profitability.""")

# Section 3: Innovation and Uniqueness of the Solution
pdf.set_font("Arial", 'B', 12)
pdf.cell(200, 10, txt="3. Innovation and Uniqueness of the Solution", ln=True)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 10, txt="""The solution stands out due to its innovative approach to integrating the supply chain, technology, and farming practices. Key innovative aspects include:

3.1 **Integrated Supply Chain Management**:
- The platform uniquely integrates middlemen into the supply chain, ensuring that farmers have a guaranteed market while allowing middlemen to profit from the sale of processed products.

3.2 **Holistic Farming Support System**:
- Beyond contract farming, the platform offers a full suite of farming tools, from weather forecasting to automation, providing comprehensive support to farmers.

3.3 **Scalability and Adaptability**:
- The platform is designed to be scalable, with the ability to add new tools and services as needed. It can be adapted to different regions and crop types, making it a versatile solution.

3.4 **Sustainability Focus**:
- The inclusion of IoT and automation technologies promotes sustainable farming practices by optimizing resource use and reducing environmental impact.""")

# Section 4: Analysis of the Feasibility of the Idea
pdf.set_font("Arial", 'B', 12)
pdf.cell(200, 10, txt="4. Analysis of the Feasibility of the Idea", ln=True)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 10, txt="""The feasibility of the platform is supported by several factors:

4.1 **Technological Availability**:
- The required technologies, such as IoT, automation, and data analytics, are readily available and can be integrated into the platform with existing infrastructure.

4.2 **Market Demand**:
- There is a strong demand for digital solutions in agriculture, driven by the need for efficiency, sustainability, and profitability.

4.3 **Government and Private Sector Support**:
- The government and private sector are increasingly investing in agricultural technology, providing a favorable environment for the implementation of the platform.

4.4 **Financial Viability**:
- The business model, which includes a profit margin for middlemen and service fees for additional tools, ensures the platform's financial sustainability.

4.5 **Ease of Adoption**:
- The platform is designed to be user-friendly, with training and support provided to ensure farmers can easily adopt and benefit from the solution.""")

# Section 5: Potential Challenges and Risks
pdf.set_font("Arial", 'B', 12)
pdf.cell(200, 10, txt="5. Potential Challenges and Risks", ln=True)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 10, txt="""While the solution is promising, there are potential challenges and risks that need to be addressed:

5.1 **Adoption Resistance**:
- Farmers may be reluctant to adopt new technologies and digital platforms, particularly in rural areas with limited exposure to modern technology.

5.2 **Infrastructure Limitations**:
- Rural areas may face challenges related to internet connectivity, electricity, and access to modern equipment, which could hinder the platform's effectiveness.

5.3 **Market Volatility**:
- The platform's success depends on stable market conditions. Fluctuations in market prices could affect the profitability of middlemen and, consequently, the platform's sustainability.

5.4 **Legal and Regulatory Issues**:
- Ensuring that contracts are legally binding and comply with local regulations could pose challenges, particularly in areas with complex or unclear legal frameworks.

5.5 **Data Privacy and Security**:
- Protecting farmers' data and ensuring secure transactions are critical, particularly given the increasing risk of cyber threats.""")

# Section 6: Strategies for Overcoming These Challenges
pdf.set_font("Arial", 'B', 12)
pdf.cell(200, 10, txt="6. Strategies for Overcoming These Challenges", ln=True)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 10, txt="""To overcome these challenges, the following strategies will be implemented:

6.1 **Education and Training Programs**:
- Conducting workshops and providing training sessions for farmers to familiarize them with the platform and the benefits of digital farming tools.

6.2 **Infrastructure Development Partnerships**:
- Collaborating with telecom providers and government agencies to improve internet connectivity and access to modern farming equipment in rural areas.

6.3 **Diversification and Risk Management**:
- Offering a diverse range of products and services on the platform to mitigate the impact of market volatility. Additionally, implementing risk management strategies such as crop insurance.

6.4 **Legal Support and Advocacy**:
- Providing legal assistance to ensure contracts are enforceable and comply with local regulations. Engaging with policymakers to advocate for supportive legal frameworks.

6.5 **Robust Data Security Measures**:
- Implementing advanced encryption and security protocols to protect user data and ensure secure transactions on the platform.""")

# Section 7: Potential Impact on the Target Audience
pdf.set_font("Arial", 'B', 12)
pdf.cell(200, 10, txt="7. Potential Impact on the Target Audience", ln=True)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 10, txt="""The platform is expected to have a significant impact on the target audience, which includes farmers, buyers, and middlemen:

7.1 **Farmers**:
- Farmers will benefit from a stable and secure income through guaranteed contracts, access to essential farming tools, and reduced operational costs.

7.2 **Buyers**:
- Buyers will have access to a consistent and reliable supply of high-quality agricultural products, sourced directly from farmers under contract.

7.3 **Middlemen**:
- Middlemen will profit from the sale of processed products at a higher price, while also playing a key role in stabilizing the agricultural market.

7.4 **Community**:
- The platform will contribute to the economic development of rural communities by increasing farmers' income and providing access to modern farming practices.""")

# Section 8: Benefits of the Solution (Social, Economic, Environmental)
pdf.set_font("Arial", 'B', 12)
pdf.cell(200, 10, txt="8. Benefits of the Solution", ln=True)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 10, txt="""The solution offers multiple benefits across social, economic, and environmental dimensions:

8.1 **Social Benefits**:
- Empowering farmers with secure and stable incomes, reducing poverty and improving living standards in rural areas.
- Enhancing the knowledge and skills of farmers through access to modern farming tools and technologies.

8.2 **Economic Benefits**:
- Boosting the profitability of the agricultural sector by increasing productivity and reducing operational costs""")

pdf_file_path = "pdf.pdf"
pdf.output(pdf_file_path)

pdf_file_path


'pdf.pdf'